In [8]:
import torch
import torch.nn as nn
import random
from datasets import load_dataset

# === Parameters ===
SEQ_LENGTH = 75
EMBED_DIM = 64
HIDDEN_DIM = 128
NUM_LAYERS = 1
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# === Load data to get vocabulary ===
print("Loading dataset and vocab...")
dataset = load_dataset('sander-wood/irishman', split='train')
texts = dataset['abc notation']
vocab = sorted(set(''.join(texts)))
char2idx = {ch: i for i, ch in enumerate(vocab)}
idx2char = {i: ch for ch, i in char2idx.items()}
VocabSize = len(vocab)

# === Define model ===
class SimpleRNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = nn.Embedding(VocabSize, EMBED_DIM)
        self.lstm = nn.LSTM(EMBED_DIM, HIDDEN_DIM, NUM_LAYERS, batch_first=True)
        self.fc = nn.Linear(HIDDEN_DIM, VocabSize)

    def forward(self, x, hidden=None):
        x = self.embed(x)
        out, hidden = self.lstm(x, hidden)
        out = out.reshape(-1, HIDDEN_DIM)
        logits = self.fc(out)
        return logits, hidden

# === Load model ===
model = SimpleRNN().to(DEVICE)
model.load_state_dict(torch.load("best_model.pth", map_location=DEVICE))
model.eval()
print("Model loaded successfully!")

# === Generate ABC tune ===
M_options = ["M:6/8", "M:4/4", "M:3/4"]
L_options = ["L:1/8", "L:1/16", "L:1/4"]
K_options = ["K:D", "K:G", "K:C"]

start = f"""X:1
T:Generated Tune
{random.choice(M_options)}
{random.choice(L_options)}
{random.choice(K_options)}
"""

seq = [char2idx.get(c, 0) for c in start]
hidden = None

for _ in range(800):  # generate 800 characters
    inp_seq = seq[-SEQ_LENGTH:]
    inp = torch.tensor(inp_seq).unsqueeze(0).to(DEVICE)
    logits, hidden = model(inp, hidden)
    probs = torch.softmax(logits[-1], dim=0)
    idx = torch.multinomial(probs, 1).item()
    seq.append(idx)

# === Decode and print ===
generated = ''.join(idx2char[i] for i in seq)
print("\n=== Generated ABC Notation ===\n")
print(generated)


Loading dataset and vocab...


C:\Users\geome\AppData\Local\Temp\ipykernel_1204\4220505250.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth", map_locat

Model loaded successfully!

=== Generated ABC Notation ===

X:1
T:Generated Tune
M:6/8
L:1/16
K:G
"G" BcB GBd |"C" c2 c cde |"G" d3 z f2 |"F" g2 f fed |"C" c2 G e2 e | efg e3 |"G#7""^4" GGG"Gm" d2 B |"D7" dcA A3 :: 
[K:C]"Em" B>BB"E" AG=F |"Ab7" BA2 c2B2 |"^A,7" e2 z2"^(E)" B4 |]"Cm" [ec]2 z2 z a2 | 
"Bbm" b3- ab!b4 b2 a2 |"C7" g3 b !>!g3 f _b2 |"F" c'a ^ga b2 a2 a2 | c'f ad' f3 a ^ge | 
"Fm" fc af ad' ag a2 | A^c ad' ba"Eb" g(3b/g/a/b ::"Bb" b>b a/=g/b/f/ g/f/g/e/ f2 Bd | 
"F7" cf Aa cA cF Fc/C/ | cc cf c2 fc c/f/f/a/ | [a,A]2 | z a f>f ff Ac | fa fa b2 bg | af cf fF Ac/f/ | bg/f/ c'>a c'd' gc' | 
 gd'bg egce | gdad fgab | gb/a/ fafd2 || |] D2 |: G4 A3d/d | c2c2d2BG | A2>A2 F2A2 | f2 F2fff2 | 
 FGAB cedc | B2G2B2 :: d2 | g2gab2c'2 | b2a2a2 b2 | c'gbg f2b2 | agab f2a2 | 
 g2g2b2gb | aaa2a2ag | a2f2a2gf | efge d2B2 | cdc2 efg2 | f2f2 a4 | agfe d2c2 | C2c2 c2c2 | 
 gfef fede | c2c2c2d2 |
